To get access to the data, follow instructions here: 

https://wiki.humanconnectome.org/display/PublicData/How+To+Connect+to+Connectome+Data+via+AWS

In [1]:
import numpy as np

In [2]:
# Install from Github: https://github.com/yeatmanlab/pyAFQ
import AFQ.data as afd
import AFQ.registration as afr

/Users/tpawlowski/workspace/dokstud/mri_pipeline/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
data_dict = afd.fetch_hcp([994273])

In [4]:
# This is a dependency of AFQ, so should already be installed at this point:
from dipy.reconst import mapmri
from dipy.core.gradients import gradient_table

In [5]:
import os.path as op

In [6]:
home = op.expanduser('~')

In [7]:
gtab = gradient_table(op.join(home, 'AFQ_data/HCP/sub-994273/sess-01/dwi/sub-994273_dwi.bval'), 
                      op.join(home, 'AFQ_data/HCP/sub-994273/sess-01/dwi/sub-994273_dwi.bvec'),
                      b0_threshold=10)

In [8]:
# These values are probably incorrect, but it doesn't matter for our purposes here:
big_delta = 0.0365  # seconds
small_delta = 0.0157  # seconds

In [9]:
# Also a dependency, so you should have it:
import nibabel as nib

In [10]:
img = nib.load(op.join(home, 'AFQ_data/HCP/sub-994273/sess-01/dwi/sub-994273_dwi.nii.gz'))

In [11]:
# This is 4D data -- the last dimension is the time dimension, the first three dimensions are spatial dimensions
data = img.get_data()

In [12]:
# Install cvxpy before running the following cell: 
#    `pip install cvxpy`

In [13]:
reference_data =  data[..., 0]
reference_img = nib.Nifti1Image(reference_data, img.affine)

In [14]:
from dipy.segment.mask import median_otsu

In [15]:
_, mask = median_otsu(reference_data, 4, 2, False, dilate=1)

In [16]:
# We'll extend this array in each iteration:
fit_data = data[..., 0, None]

In [ ]:
for idx in np.arange(1, 288):
    print("Incoming data: ", idx)
    this_data = data[..., idx]

    print("Register incoming data to the reference image")
    this_data, reg_affine = afr.affine_registration(this_data, 
                                                    reference_data)
    
    print("Create full series for this iteration")
    fit_data = np.concatenate([fit_data, this_data[..., None]], axis=-1)
    
    if idx > 20:
        print("Create the gradient table for this iteration")
        this_gtab = gradient_table(bvals=gtab.bvals[:idx + 1], 
                                   bvecs=gtab.bvecs[:idx + 1],
                                   big_delta=big_delta,
                                   small_delta=small_delta, 
                                   b0_threshold=10)

        print("Initialize model")
        this_model = mapmri.MapmriModel(this_gtab, 
                                        radial_order=6,
                                        laplacian_regularization=True,
                                        laplacian_weighting=.05,
                                        positivity_constraint=True)

        print("Fit the model in each voxel")
        this_model.fit(fit_data, mask=mask)

Incoming data:  1
Register incoming data to the reference image
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Create full series for this iteration
Incoming data:  2
Register incoming data to the reference image
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Create full series for this iteration
Incoming data:  3
Register incoming data to the reference image
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing 

Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Create full series for this iteration
Incoming data:  21
Register incoming data to the reference image
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Create full series for this iteration
Create the gradient table for this iteration
Initialize model
Fit the model in each voxel
